### Se van a normalizar los datos y a reordenar las filas

In [1]:
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
# Diccionario de prueba
a = {
    'col1': [1,2,3,4,5,6,7,8,9,0],
    'col2': [4,5,6,5,6,7,8,2,3,4]
}

df = pd.DataFrame(a)
df

,col1,col2
0,1,4
1,2,5
2,3,6
3,4,5
4,5,6
5,6,7
6,7,8
7,8,2
8,9,3
9,0,4


In [3]:
columnas = []
profundidades = ['_0','_5','_10']

# añado etiqueta de profundidad
for i in range(3):
    columnas_aux = df.columns + profundidades[i]
    [columnas.append(x) for x in columnas_aux]
    
df_2 = pd.DataFrame(columns=columnas)
df_2

,col1_0,col2_0,col1_5,col2_5,col1_10,col2_10


In [4]:
# concateno datos de la tres profundidades en una sola
for cont in range(int(df.shape[0]/3)):
    inicio = cont*3
    df_aux = df.loc[inicio:inicio+2]
    print(df_aux)
    lista = []

    for fila in df_aux.values.tolist():
        for elem in fila :
            lista.append(elem)
    print(lista)
    df_2.loc[cont] = lista
    

   col1  col2
0     1     4
1     2     5
2     3     6
[1, 4, 2, 5, 3, 6]
   col1  col2
3     4     5
4     5     6
5     6     7
[4, 5, 5, 6, 6, 7]
   col1  col2
6     7     8
7     8     2
8     9     3
[7, 8, 8, 2, 9, 3]


In [5]:
df_2

,col1_0,col2_0,col1_5,col2_5,col1_10,col2_10
0,1,4,2,5,3,6
1,4,5,5,6,6,7
2,7,8,8,2,9,3


# Con estructura avistamientos

In [6]:
# Estructura quitando las columnas de las coordenadas pues no nos sirven
df = pd.read_excel('../4EstructuraFinalSinMissings.xlsx')
df = df.fillna(method='ffill', axis=0).set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad']).drop(['level_0','index'],axis = 1)
# Eliminar columnas Coords
cols = [c for c in df.columns if 'C' not in c]
df=df[cols]

df

mlotst  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-42.666667 -74.083333 2015-05-27 1.0           0.494025     49.592583   
                                               5.078224     49.592583   
                                               9.572997     49.592583   
-41.833333 -73.833333 2014-06-04 1.0           0.494025      9.613331   
                                               5.078224      9.613331   
...                                                               ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     10.528886   
                                               9.572997     10.528886   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     14.954070   
                                               5.078224     14.954070   
                                               9.572997     14.954070   

                                                                 zos  \
Latitud    Longitud   Fecha      Avistamientos Profundidad             
-42.666667 -74.083333 2015-05-27 1.0           0.494025     0.094607   
                                               5.078224     0.094607   
                                               9.572997     0.094607   
-41.833333 -73.833333 2014-06-04 1.0           0.494025     0.078127   
                                               5.078224     0.078127   
...                                                              ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     0.025636   
                                               9.572997     0.025636   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     0.069277   
                                               5.078224     0.069277   
                                               9.572997     0.069277   

                                                              bottomT  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-42.666667 -74.083333 2015-05-27 1.0           0.494025     12.819330   
                                               5.078224     12.819330   
                                               9.572997     12.819330   
-41.833333 -73.833333 2014-06-04 1.0           0.494025     11.473098   
                                               5.078224     11.473098   
...                                                               ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     14.183142   
                                               9.572997     14.183142   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     15.737389   
                                               5.078224     15.737389   
                                               9.572997     15.737389   

                                                               thetao  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-42.666667 -74.083333 2015-05-27 1.0           0.494025     12.818598   
                                               5.078224     12.817133   
                                               9.572997     12.815668   
-41.833333 -73.833333 2014-06-04 1.0           0.494025     11.325144   
                                               5.078224     11.443800   
...                                                               ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     15.145573   
                                               9.572997     14.855526   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     15.744713   
                                               5.078224     15.731529   
                                               9.572997     15.716880   

                                                                   so  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-42.666667 -74.083333 2015-05-27 1.0           0.494025     33.338421   
                                               5.078224  

In [7]:
columnas = []
profundidades = ['_0','_5','_10']

# añado etiqueta de profundidad
def resetea_atributos():
    for i in range(3):
        columnas_aux = df.columns + profundidades[i]
        [columnas.append(x) for x in columnas_aux]

    df_atributos = pd.DataFrame(columns=columnas)
    return df_atributos
# df_atributos

In [8]:
df_atributos = resetea_atributos()
# meto tres profundidades consecutivas
total_lineas = df.shape[0]/3
pbar = tqdm(total = total_lineas)
for cont in range(int(df.shape[0]/3)):
    pbar.update(1)
    inicio = cont*3
    df_aux = df.iloc[inicio:inicio+3]
    lista = list()

    for fila in df_aux.values.tolist():
        for elem in fila :
            lista.append(elem)
            
    dict_atr = dict(zip(df_atributos.columns,lista))
    df_atr_aux = pd.DataFrame(dict_atr,index=[0])
    df_atributos = pd.concat([df_atributos,df_atr_aux])
    
df_atributos

,mlotst_0,zos_0,bottomT_0,thetao_0,so_0,uo_0,vo_0,Profundidad_izq_1_0,mlotst_izq_1_0,zos_izq_1_0,...,uo_inf_2.1_10,vo_inf_2.1_10,Profundidad_inf_2.2_10,mlotst_inf_2.2_10,zos_inf_2.2_10,bottomT_inf_2.2_10,thetao_inf_2.2_10,so_inf_2.2_10,uo_inf_2.2_10,vo_inf_2.2_10
0,49.592583,0.094607,12.819330,12.818598,33.338421,0.036012,0.039064,0.494025,51.576283,0.095523,...,0.007935,0.043336,9.572997,51.576283,0.097049,10.456466,12.938719,33.420818,0.006104,0.032350
0,9.613331,0.078127,11.473098,11.325144,33.407085,0.380261,0.097659,0.494025,8.697776,0.082705,...,-0.032960,-0.161138,9.572997,11.291849,0.098575,11.260689,11.218207,33.396404,-0.107425,-0.066530
0,14.648885,0.196234,9.668355,12.579089,33.419292,0.327769,-0.048219,0.494025,10.528886,0.096438,...,0.017090,0.134281,9.572997,37.842953,0.102847,12.368145,12.324198,33.307899,-0.036622,0.098880
0,18.616293,0.090945,12.857417,12.826654,33.338421,0.040284,0.006104,0.494025,23.499254,0.090640,...,-0.012818,0.006714,9.572997,41.047398,0.050661,12.315409,12.303690,33.371990,0.020753,0.118412
0,22.125921,0.077517,12.760735,12.757072,33.349102,0.029908,0.073244,0.494025,29.145180,0.077822,...,-0.004273,0.067751,9.572997,41.047398,0.024110,12.188696,12.187963,33.359783,-0.003052,0.115360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,10.528886,0.214850,15.272285,17.710592,34.873501,-0.057985,0.126957,0.494025,10.528886,0.218818,...,-0.034791,-0.045778,9.572997,12.207404,0.221564,4.393291,20.789057,35.168003,-0.020142,-0.067141
0,10.528886,0.145268,14.262978,17.144413,34.867397,-0.091556,0.131840,0.494025,10.528886,0.152593,...,-0.059206,0.159917,9.572997,10.834071,0.174566,4.266579,19.529251,35.006256,-0.007324,0.103153
0,10.528886,-0.040590,13.032472,14.162633,34.774315,-0.068972,0.065920,0.494025,10.528886,-0.037843,...,-0.081179,0.186773,9.572997,10.528886,-0.020142,3.884243,14.838679,34.888760,-0.058596,0.155034
0,10.528886,0.025636,14.183142,15.312571,34.765160,-0.016480,0.082400,0.494025,10.528886,0.027772,...,0.023804,-0.009766,9.572997,12.665182,0.023804,3.858608,15.999603,34.853664,0.020753,-0.030519


## normalizar

In [9]:
# sin uso de bibliotecas 
df = df_atributos.copy()
normalized_df=(df-df.min())/(df.max()-df.min())
print(df.mlotst_0.max())
normalized_df

152.1347808837891


,mlotst_0,zos_0,bottomT_0,thetao_0,so_0,uo_0,vo_0,Profundidad_izq_1_0,mlotst_izq_1_0,zos_izq_1_0,...,uo_inf_2.1_10,vo_inf_2.1_10,Profundidad_inf_2.2_10,mlotst_inf_2.2_10,zos_inf_2.2_10,bottomT_inf_2.2_10,thetao_inf_2.2_10,so_inf_2.2_10,uo_inf_2.2_10,vo_inf_2.2_10
0,0.296335,0.366700,0.462185,0.172787,0.523447,0.478261,0.356818,NaN,0.294549,0.354188,...,0.515190,0.608955,NaN,0.284836,0.340812,0.735232,0.166180,0.350558,0.529560,0.509585
0,0.021990,0.339537,0.360498,0.042349,0.534854,1.000000,0.429545,NaN,0.000000,0.332472,...,0.430380,0.408955,NaN,0.014344,0.343483,0.803077,0.013816,0.344972,0.295597,0.423323
0,0.056544,0.534205,0.224177,0.151868,0.536882,0.920444,0.248485,NaN,0.012579,0.355739,...,0.534177,0.697910,NaN,0.192623,0.350962,0.896503,0.111760,0.324720,0.441509,0.567625
0,0.083770,0.360664,0.465062,0.173490,0.523447,0.484736,0.315909,NaN,0.101677,0.345915,...,0.472152,0.573134,NaN,0.214139,0.259615,0.892054,0.109944,0.339386,0.559748,0.584665
0,0.107853,0.338531,0.457759,0.167413,0.525222,0.469010,0.399242,NaN,0.140461,0.324199,...,0.489873,0.632836,NaN,0.214139,0.213141,0.881364,0.099695,0.336592,0.510692,0.582002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.028272,0.564889,0.647469,0.600051,0.778454,0.335800,0.465909,NaN,0.012579,0.563082,...,0.426582,0.521791,NaN,0.020492,0.558761,0.223739,0.861387,0.750349,0.475472,0.422790
0,0.028272,0.450201,0.571231,0.550601,0.777440,0.284921,0.471970,NaN,0.012579,0.450879,...,0.375949,0.722985,NaN,0.011270,0.476496,0.213050,0.749822,0.713338,0.501887,0.571352
0,0.028272,0.143863,0.478285,0.290174,0.761977,0.319149,0.390152,NaN,0.012579,0.128232,...,0.330380,0.749254,NaN,0.009221,0.135684,0.180796,0.334436,0.686452,0.396226,0.616613
0,0.028272,0.253018,0.565201,0.390609,0.760456,0.398705,0.410606,NaN,0.012579,0.239400,...,0.548101,0.557015,NaN,0.023566,0.212607,0.178633,0.437245,0.678422,0.559748,0.454739


In [10]:
49.592583 / 147.2518157958984

0.336787582088216

In [11]:
n=(1-0)/(2-0)
n=(1--1)/(2--1)
n=(1--1)/(1--1)

n

1.0

In [12]:
# con scikit learn ejemplos
from sklearn import preprocessing
import numpy as np

X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
X_normalized = preprocessing.normalize(X, norm='l2')

X_normalized



array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

In [13]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
X_normalized = preprocessing.normalize(X, norm='l1')

X_normalized

array([[ 0.25, -0.25,  0.5 ],
       [ 1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.5 , -0.5 ]])

In [14]:
# con estructura
X = df_atributos.values.tolist()
X_normalized = preprocessing.normalize(X, norm='l2')

X_normalized

df_norm = pd.DataFrame(X_normalized,columns=df_atributos.columns)
df_norm.to_excel('dfAtributosNormalizado.xlsx')

## listado avistamientos

In [15]:
df = pd.read_excel('../4EstructuraFinalSinMissings.xlsx')
df = df.fillna(method='ffill', axis=0).drop(['level_0','index'],axis = 1)
listado_avistamientos = df.Avistamientos.values.tolist()
avistamientos = [x for x in listado_avistamientos[::3]]
df_avistamientos = pd.DataFrame(avistamientos,columns=['Avistamientos'])
df_avistamientos.to_excel('dfAvistamientos.xlsx')